In [43]:
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from Code.UtilityFunctions.get_data_path import get_path

In [3]:
def wikidata_query(sparql_query: str):
    # From https://www.wikidata.org/wiki/Wikidata:SPARQL_query_service/queries/examples#Cats
    user_agent = "Location Information - USA & Canada/%s.%s" % (sys.version_info[0], sys.version_info[1])
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql", agent=user_agent)
    sparql.setQuery(sparql_query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    results_df = pd.json_normalize(results['results']['bindings'])
    return results_df

# Cities in the USA

In [30]:
# P31 instance of
# P131 located in the administrative territorial entity
# P279 subclass of
# P585 point in time
# P1082 population
# Q515 city
# Q35657 U.S. state

query_us_cities = """
SELECT ?city ?cityLabel ?county ?population
WHERE {
    {?city wdt:P31/wdt:P279* wd:Q532}
    UNION
    {?city wdt:P31/wdt:P279* wd:Q515}
	?city wdt:P17 wd:Q30 .
	OPTIONAL { ?city wdt:P131 ?county . }
	?city p:P1082 ?statement .
	?statement ps:P1082 ?population .
	?statement pq:P585 ?date .
	FILTER NOT EXISTS {
		?city p:P1082/pq:P585 ?date2 .
		FILTER(?date2 > ?date)
	}
	SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en" . }
}
"""

result_us_cities = wikidata_query(query_us_cities)
result_us_cities = result_us_cities[["city.value", "cityLabel.value", "county.value", "population.value"]]
us_city_renames = {"city.value": "city_iri", "cityLabel.value": "city", "county.value": "county_iri", "population.value": "population"}
result_us_cities.rename(columns=us_city_renames, inplace=True)
# result_us_cities.to_csv(path_or_buf=get_path("usaCities.csv"), index=False)

In [31]:
# 19,495
len(result_us_cities)

15835

# Counties in the USA

In [25]:
query_us_counties = """
SELECT DISTINCT ?county ?countyLabel ?state
WHERE {
    ?county wdt:P31/wdt:P279* wd:Q6256 .
    ?county wdt:P131 ?state .
    ?state wdt:P31 wd:Q35657 .
    ?county wdt:P17 wd:Q30 .
    SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en" . }
}
"""

query_us_counties = """
SELECT ?county ?countyLabel ?state
WHERE {
	?county wdt:P31/wdt:P279* wd:Q47168 .
	?county wdt:P131 ?state .
    SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en" . }
}
"""

result_us_counties = wikidata_query(query_us_counties)
result_us_counties = result_us_counties[["county.value", "countyLabel.value", "state.value"]]
us_county_renames = {"county.value": "county_iri", "countyLabel.value": "county", "state.value": "state_iri"}
result_us_counties.rename(columns=us_county_renames, inplace=True)
# result_us_cities.to_csv(path_or_buf=get_path("usaCounties.csv"), index=False)

In [26]:
# 3,143
len(result_us_counties)

,county_iri,county,state_iri
0,http://www.wikidata.org/entity/Q16861,Bexar County,http://www.wikidata.org/entity/Q1439
1,http://www.wikidata.org/entity/Q26502,Tyler County,http://www.wikidata.org/entity/Q1439
2,http://www.wikidata.org/entity/Q26506,Robertson County,http://www.wikidata.org/entity/Q1439
3,http://www.wikidata.org/entity/Q26519,Hidalgo County,http://www.wikidata.org/entity/Q1439
4,http://www.wikidata.org/entity/Q26526,Kimble County,http://www.wikidata.org/entity/Q1439
...,...,...,...
3075,http://www.wikidata.org/entity/Q504445,Benton County,http://www.wikidata.org/entity/Q1415
3076,http://www.wikidata.org/entity/Q505299,Taylor County,http://www.wikidata.org/entity/Q1428
3077,http://www.wikidata.org/entity/Q505310,Tattnall County,http://www.wikidata.org/entity/Q1428
3078,http://www.wikidata.org/entity/Q544539,Haralson County,http://www.wikidata.org/entity/Q1428


# States in the USA

In [8]:
# States in US
# P31 instance of
# Q35657 U.S. state

query_all_us_states = """
SELECT ?state ?stateLabel ?country
WHERE {
  ?state wdt:P31 wd:Q35657 .
  ?state wdt:P17 ?country .
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en" . }
}
"""

result_us_states = wikidata_query(query_all_us_states)
result_us_states = result_us_states[["state.value", "stateLabel.value", "country.value"]]
us_state_renames = {"state.value": "state_iri", "stateLabel.value": "state", "country.value": "country_iri"}
result_us_states.rename(columns=us_state_renames, inplace=True)
# result_us_states.to_csv(path_or_buf=get_path("usaStates.csv"), index=False)

In [16]:
# 50
len(result_us_states)

50

# Cities in Canada

In [54]:
# Cities in Canada
# P131 located in the administrative territorial entity
# P17 country
# Q16 Canada

query_canada_cities = """
SELECT ?city ?cityLabel ?province ?population
WHERE {
    ?city wdt:P31/wdt:P279* wd:Q515 .
    {
        ?city wdt:P131 ?county .
        ?county wdt:P131 ?province .
        ?province wdt:P31 wd:Q11828004 .
    }
    UNION
    {
        ?city wdt:P131 ?province .
        ?province wdt:P31 wd:Q11828004 .
    }
    ?city p:P1082 ?statement .
    ?statement ps:P1082 ?population .
    ?statement pq:P585 ?date .
    FILTER NOT EXISTS {
        ?city p:P1082/pq:P585 ?date2 .
        FILTER(?date2 > ?date)
    }
    SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en" . }
}
"""

query_canada_cities = """
SELECT DISTINCT ?city ?cityLabel
WHERE {
	{
        ?city wdt:P31/wdt:P279* wd:Q515 .
	    ?city wdt:P131+ wd:Q1951 .
    }
	UNION
	{
        ?city wdt:P31/wdt:P279* wd:Q532 .
        ?city wdt:P131+ wd:Q1951 .
    }
    SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en" . }
}
"""


result_can_cities = wikidata_query(query_canada_cities)
# result_can_cities = result_can_cities[["city.value", "cityLabel.value", "state.value", "population.value"]]
# can_city_renames = {"city.value": "city_iri", "cityLabel.value": "city", "state.value": "state_iri", "population.value": "population"}
# result_can_cities.rename(columns=can_city_renames, inplace=True)
# result_can_cities.to_csv(path_or_buf=get_path("canadaCities.csv"))

EndPointInternalError: EndPointInternalError: The endpoint returned the HTTP status code 500. 

Response:
b'SPARQL-QUERY: queryStr=\nSELECT DISTINCT ?city ?cityLabel\nWHERE {\n\t{\n        ?city wdt:P31/wdt:P279* wd:Q515 .\n\t    ?city wdt:P131+ wd:Q1951 .\n    }\n\tUNION\n\t{\n        ?city wdt:P31/wdt:P279* wd:Q532 .\n        ?city wdt:P131+ wd:Q1951 .\n    }\n    SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en" . }\n}\n\njava.util.concurrent.TimeoutException\n\tat java.util.concurrent.FutureTask.get(FutureTask.java:205)\n\tat com.bigdata.rdf.sail.webapp.BigdataServlet.submitApiTask(BigdataServlet.java:292)\n\tat com.bigdata.rdf.sail.webapp.QueryServlet.doSparqlQuery(QueryServlet.java:678)\n\tat com.bigdata.rdf.sail.webapp.QueryServlet.doGet(QueryServlet.java:290)\n\tat com.bigdata.rdf.sail.webapp.RESTServlet.doGet(RESTServlet.java:240)\n\tat com.bigdata.rdf.sail.webapp.MultiTenancyServlet.doGet(MultiTenancyServlet.java:273)\n\tat javax.servlet.http.Htt

In [ ]:
# 7,986
result_can_cities.sort_values(by="cityLabel.value")

,city.type,city.value,cityLabel.xml:lang,cityLabel.type,cityLabel.value
263,uri,http://www.wikidata.org/entity/Q341968,en,literal,Acme
238,uri,http://www.wikidata.org/entity/Q4688739,en,literal,"Aetna, Alberta"
273,uri,http://www.wikidata.org/entity/Q408537,en,literal,Airdrie
313,uri,http://www.wikidata.org/entity/Q2327744,en,literal,"Alberta Beach, Alberta"
227,uri,http://www.wikidata.org/entity/Q4713337,en,literal,"Alcomdale, Alberta"
181,uri,http://www.wikidata.org/entity/Q4713616,en,literal,"Alder Flats, Alberta"
217,uri,http://www.wikidata.org/entity/Q4713757,en,literal,"Aldersyde, Alberta"
228,uri,http://www.wikidata.org/entity/Q4724422,en,literal,"Alhambra, Alberta"
299,uri,http://www.wikidata.org/entity/Q4727494,en,literal,Alix
202,uri,http://www.wikidata.org/entity/Q4736185,en,literal,"Altario, Alberta"


# Municipalities of Canada

In [10]:
query_canada_municipalities = """
SELECT DISTINCT ?municipality ?municipalityLabel ?province
WHERE {
    ?municipality wdt:P31/wdt:P279* wd:Q15284 .
    ?municipality wdt:P131 ?province .
    ?province wdt:P31 wd:Q11828004 .
    SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en" . }
}
"""
res = wikidata_query(query_canada_municipalities)

In [19]:
# 3,573
len(res)

1819

# Provinces of Canada

In [12]:
# Provinces of Canada
# P17 country
# Q11828004 Canada

query_all_canada_provinces = """
SELECT ?province ?provinceLabel ?country
WHERE {
  ?province wdt:P31 wd:Q11828004 .
  ?province wdt:P17 ?country .
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en" . }
}
"""

result_can_provinces = wikidata_query(query_all_canada_provinces)
result_can_provinces = result_can_provinces[["province.value", "provinceLabel.value", "country.value"]]
can_province_renames = {"province.value": "state_iri", "provinceLabel.value": "state", "country.value": "country_iri"}
result_can_provinces.rename(columns=can_province_renames, inplace=True)
# result_can_provinces.to_csv(path_or_buf=get_path("canadaProvinces.csv"))

In [17]:
# 10
len(result_can_provinces)

10